In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from audtorch.metrics.functional import pearsonr
from torch.utils.data import DataLoader, Dataset
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler

# 自定义卷积核(特征提取)

In [2]:
'''
过去d天X值构成的时序数列和Y值构成的时序数列的相关系数
'''

class ts_corr(nn.Module):
    def __init__(self, d=10, stride=10):
        super(ts_corr, self).__init__()
        self.d = d
        self.stride = stride

    def forward(self, X): #X:3D
        B, n, T = X.shape # 批量大小，特征数量，时间窗口长度
        
        w = (T - self.d) // self.stride + 1  # 窗口数量,例如T=10，d=3，stride=2时，w=4
        h = n * (n - 1) // 2  # 特征对的数量 C(n, 2)

        # 使用 unfold 提取滑动窗口 [形状: (B, n, w, d)]
        unfolded_X = X.unfold(2, self.d, self.stride)
        
        #生成C(n,2)组合数
        #例如当n=3时，rows = tensor([0,0,1]), cols = tensor([1,2,2])
        rows, cols = torch.triu_indices(n, n, offset=1)

        # 提取特征对数据得到x和y [形状: (B, h, w, d)]，分别对应batch维度，特征维度，窗口维度，时间维度
        #x为([[:,0,:,:],[:,0,:,:],[:,1,:,:])
        #y为([[:,1,:,:],[:,2,:,:],[:,2,:,:])
        
        x = unfolded_X[:, rows, :, :]
        y = unfolded_X[:, cols, :, :]
        
        x_mean = torch.mean(x, dim=3, keepdim=True) #keepdim维持原本维度不变,在维度3做mean
        y_mean = torch.mean(y, dim=3, keepdim=True)
        
        cov = ((x-x_mean)*(y-y_mean)).sum(dim=3) #(B, h, w)
        corr = cov / (torch.std(x, dim=3) * torch.std(y, dim=3)+ 1e-8) #分母添加极小值防止除零错误

        return corr

In [3]:
'''
过去d天X值构成的时序数列和Y值构成的时序数列的协方差
'''

class ts_cov(nn.Module):
    def __init__(self, d=10, stride=10):
        super(ts_cov, self).__init__()
        self.d = d
        self.stride = stride

    def forward(self, X):
        B, n, T = X.shape 
        
        w = (T - self.d) // self.stride + 1  
        h = n * (n - 1) // 2  

        unfolded_X = X.unfold(2, self.d, self.stride)
        
        rows, cols = torch.triu_indices(n, n, offset=1)

        x = unfolded_X[:, rows, :, :]
        y = unfolded_X[:, cols, :, :]
        
        x_mean = torch.mean(x, dim=3, keepdim=True)
        y_mean = torch.mean(y, dim=3, keepdim=True)
        
        cov = ((x-x_mean)*(y-y_mean)).sum(dim=3) 

        return cov

In [4]:
'''
过去d天X值构成的时序数列的标准差
'''

class ts_stddev(nn.Module):
    
    def __init__(self, d = 10, stride = 10):
        super(ts_stddev,self).__init__()
        self.d = d
        self.stride = stride
        
    def forward(self, X):
        #input:(B,n,T)，在T维度用unfold展开窗口，变为(B,n,w,d),w为窗口数量会自动计算
        unfolded_X = X.unfold(2, self.d, self.stride)
        #在每个窗口，即d维度上进行std计算
        std = torch.std(unfolded_X, dim=3) #输出形状为(B,n,w)
        
        return std

In [5]:
'''
过去d天X值构成的时序数列的平均值除以标准差
'''

class ts_zscore(nn.Module):
    
    def __init__(self, d = 10, stride = 10):
        super(ts_zscore,self).__init__()
        self.d = d
        self.stride = stride
        
    def forward(self, X):
        
        unfolded_X = X.unfold(2, self.d, self.stride)
        
        mean = torch.mean(unfolded_X, dim=3)
        std = torch.std(unfolded_X, dim=3)
        zscore = mean / (std + 1e-8)
        
        return zscore

In [6]:
'''
研报原话为：
(X - delay(X, d))/delay(X, d)-1, delay(X, d)为 X 在 d 天前的取值
这里可能有误，return为“收益率“，应该是误加了-1
为了保持代码一致性，这里计算的是(X - delay(X, d-1))/delay(X, d-1),  delay(X, d-1)为 X 在 d-1 天前的取值
在构造卷积核的逻辑上是相似的
'''

class ts_return(nn.Module):
    
    def __init__(self, d = 10, stride = 10):
        super(ts_return,self).__init__()
        self.d = d
        self.stride = stride
        
    def forward(self, X):
        
        unfolded_X = X.unfold(2, self.d, self.stride)
        return1 = unfolded_X[:,:,:,-1] /(unfolded_X[:,:,:,0] + 1e-8) - 1
        
        return return1

In [7]:
'''
过去d天X值构成的时序数列的加权平均值，权数为d, d – 1, …, 1(权数之和应为1，需进行归一化处理）
其中离现在越近的日子权数越大。 
'''

class ts_decaylinear(nn.Module):
    
    def __init__(self, d = 10, stride = 10):
        super(ts_decaylinear,self).__init__()
        self.d = d
        self.stride = stride
        #如下设计的权重系数满足离现在越近的日子权重越大
        weights = torch.arange(d, 0, -1, dtype = torch.float32) 
        weights = weights / weights.sum()
        #注册权重，不用在前向传播函数中重复计算
        #注册了一个形状为(d,)的一维张量，存放权重系数，以便在forward函数中使用
        self.register_buffer('weights', weights) 
        
    def forward(self, X):
        
        unfolded_X = X.unfold(2, self.d, self.stride)
        #view将一维张量广播为4D张量，并在时间维度上，将weights与unfoled_X相乘
        decaylinear = torch.sum(unfolded_X * self.weights.view(1,1,1,-1), dim=-1)
        
        return decaylinear

# 神经网络结构设计 

原始路径(RawPath)：特征提取层→BN

池化路径(PoolPath)：特征提取层→池化层→BN

展平→全连接层→预测目标

In [8]:
'''
原始路径：特征提取层+BN
'''
class RawPath(nn.Module):
    def __init__(self, extractor, bn_dim): #传入参数：卷积核，特征维度
        super().__init__()
        self.extractor = extractor
        self.bn = nn.BatchNorm1d(bn_dim)
        
    def forward(self, X):
        x = self.extractor(X) #extract
        x = self.bn(x) #BN

        return x

In [9]:
'''
池化路径：特征提取层+池化层+BN
'''
class PoolPath(nn.Module): #传入参数：卷积核，特征维度
    def __init__(self, extractor, bn_dim, d_pool=3, s_pool=3):
        super().__init__()
        self.extractor = extractor
        
        self.avg_pool = nn.AvgPool1d(d_pool, s_pool)
        self.max_pool = nn.MaxPool1d(d_pool, s_pool)
        
        #每个池化操作bihv使用独立的 BatchNorm 层
        #否则会导致三种不同统计量（均值、最大值、最小值）的分布被强制归一化到同一参数
        self.bn_avg = nn.BatchNorm1d(bn_dim) 
        self.bn_max = nn.BatchNorm1d(bn_dim)
        self.bn_min = nn.BatchNorm1d(bn_dim)

    def forward(self, X):
        x = self.extractor(X)
        
        x_avg = self.bn_avg(self.avg_pool(x))
        x_max = self.bn_max(self.max_pool(x))
        x_min = self.bn_min(-self.max_pool(-x))#手动取反实现min_pool
        
        return torch.cat([x_avg, x_max, x_min], dim = 1) #在特征维度进行拼接

In [10]:
class AlphaNet(nn.Module):

    def __init__(self, d=10, stride=10, d_pool=3, s_pool=3, n=9, T=30): #池化层窗口d=3，步长stride=3
        super(AlphaNet, self).__init__()
        
        self.d = d 
        self.stride = stride 
        h = n * (n - 1) // 2 #手动计算cov和corr特征提取后的特征维度大小
        w = (T - d) // stride + 1 #手动计算特征提取层窗口数
        w_pool = (w - d_pool) // s_pool + 1 #手动计算池化层窗口数
        
        #特征提取层列表，共7个
        self.extractors = nn.ModuleList([
            ts_corr(d,stride),
            ts_cov(d,stride),
            ts_stddev(d,stride),
            ts_zscore(d,stride),
            ts_return(d,stride),
            ts_decaylinear(d,stride),
            nn.AvgPool1d(d,stride) #原研报中的ts_mean
        ])
        

        # 初始化双路径
        self.raw_paths = nn.ModuleList()
        self.pool_paths = nn.ModuleList()
        
        # 前两个特征提取器使用h维BN
        for i in range(2):
            self.raw_paths.append(RawPath(self.extractors[i], h))
            self.pool_paths.append(PoolPath(self.extractors[i], h))
        
        # 后五个特征提取器使用n维BN
        for i in range(2, 7):
            self.raw_paths.append(RawPath(self.extractors[i], n))
            self.pool_paths.append(PoolPath(self.extractors[i], n))

        raw_dim = (2*h + 5*n)*w #计算初始路径展平后的维度
        pooled_dim = (h*2*3 + n*5*3)*w_pool #计算池化路径展平后的维度

        self.head = nn.Sequential(
            nn.Linear(raw_dim + pooled_dim, 30),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(30, 1)
        )

        # 初始化
        self.initialize_weights()

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                
                #截断正态初始化
                fan_in = m.weight.size(1)
                std = math.sqrt(1.0 / fan_in)  # Xavier方差标准
                nn.init.trunc_normal_(m.weight, std=std, a=-2*std, b=2*std)
                nn.init.normal_(m.bias, std=1e-6)
                
                #Kaiming初始化
                #nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                
                #Xavier初始化
                #nn.init.xavier_uniform_(m.weight)
                #nn.init.normal_(m.bias, std=1e-6)
            
    def forward(self, X):
        
        raw_features = [path(X).flatten(1) for path in self.raw_paths] #原始路径得到的张量进行展平
        pool_features = [path(X).flatten(1) for path in self.pool_paths] #池化路径得到的张量进行展平
        all_features = torch.cat(raw_features + pool_features, dim=1) 
        
        return self.head(all_features)

# 数据准备

In [11]:
X = np.load('npy_v1/X_fe.npy')
Y = np.load('npy_v1/Y_fe.npy')
dates = np.load('npy_v1/Y_dates.npy')

print('Shape of X: ', X.shape)
print('Shape of Y: ', Y.shape)

Shape of X:  (930008, 9, 30)
Shape of Y:  (930008,)


In [12]:
'''
对X进行窗口标准化
'''
class myDataset(Dataset):
    '''
    自定义数据集，将原始数据从 numpy arrays 转换成 float 格式的 tensors
    '''
    
    def __init__(self, X, y, scaler = None, is_train = True):
        super(myDataset, self).__init__()
        self.y = y.reshape(-1, 1)
        
        self.origin_shape = X.shape
        
        # (B, n, T) → (B*T, n)
        X_2d = X.reshape(-1, self.origin_shape[1]) 
        
        #训练模式，同时完成 拟合（计算均值和标准差） 和 转换（应用标准化）
        if is_train: 
            self.scaler = StandardScaler()
            X_trans = self.scaler.fit_transform(X_2d)
            #X_trans = np.clip(X_trans, -5, 5)  # 限制标准化后的值在±5个标准差内
        
        #验证/测试模式，仅进行 转换（应用标准化），不重新计算均值和标准差
        #预先计算好的均值和标准差存储在标准化器（StandardScaler）的内部属性中
        
        else: 
            self.scaler = scaler
            X_trans = self.scaler.transform(X_2d)
            
        self.X = X_trans.reshape(self.origin_shape)
        self.X = torch.as_tensor(self.X, dtype=torch.float32)
        self.y = torch.as_tensor(self.y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def get_scaler(self):
        return self.scaler

In [13]:
#alphanet = AlphaNet(d=10, stride=10, n=9)

In [14]:
#alphanet(torch.tensor(X[:5]).float())

In [15]:
target_dates = np.array([datetime.strptime(str(date), '%Y-%m-%d').date() for date in dates])
unique_dates = sorted(np.unique(target_dates))

In [16]:
#用于得到不同的轮次,确保每个轮次为1626天

start_dates = []
starts, valid_starts, test_starts, ends,  = [], [], [], []

i, start, end = 0, 0, 0

k = int(1500*0.5) #按1：1划分训练集和测试集

while i + 1500 + 126 <= len(unique_dates):
    start_dates.append(i)
    
    start = sum(target_dates < unique_dates[i])
    starts.append(start)
    
    valid_start = sum(target_dates < unique_dates[i+k]) #训练集终点
    valid_starts.append(valid_start)
    
    test_start = sum(target_dates < unique_dates[i+1500]) #验证集重点（1500天）
    test_starts.append(test_start)
    
    end = sum(target_dates < unique_dates[i+1500+126]) #测试集终点（再126天）
    ends.append(end)
    
    i += 126

# 训练

In [ ]:
'''
只对X进行窗口标准化
'''
#### 设置随机种子，保证训练结果一致
torch.manual_seed(42)
#torch.cuda.manual_seed_all(42)

# 设置训练参数：学习率、训练迭代次数、批量大小
lr = 0.0001
n_epoch = 100
batch_size = 1000
device = torch.device("cpu")

model_name = 'alphanet'

# 初始化一个字典，用来储存模型训练期间的表现
results = {}
results['round'] = []
results['train'] = []
results['valid'] = []
results['test'] = []

# 维护 cnt 变量，记录当前是第几个训练轮次
cnt = 0

# 滚动窗口
for start, valid_start, test_start, end in zip(starts, valid_starts, test_starts, ends):
    
    # 初始化损失函数和优化器
    net = AlphaNet(d=10, stride=10, n=9)
    
    criterion = nn.MSELoss()
    optimizer = optim.RMSprop(net.parameters(), lr=lr)
    
    # 划分集
    train_set = myDataset(X[start:valid_start], Y[start:valid_start], is_train=True)
    train_scaler = train_set.get_scaler()
    
    valid_set = myDataset(X[valid_start:test_start], Y[valid_start:test_start], scaler = train_scaler, is_train=False)
    
    test_set = myDataset(X[test_start:end], Y[test_start:end], scaler = train_scaler, is_train=False)

    # 创建loader
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    # 当前训练轮次的模型储存地址
    model_path = 'Models_v1/' + model_name + '_' + str(cnt) + '.pt'
    
    count = 0
    train_loss_lst, valid_loss_lst = [], []
    best_valid_loss = float('inf')
    patience = 10
    
    for epoch in range(n_epoch):
        
        # 训练
        net.train()
        train_loss = 0
        total_samples = 0
        for x, y in tqdm(train_loader):
            x, y = x.to(device), y.to(device)
            preds = net(x)
            loss = criterion(preds, y)
            train_loss += loss.item() * x.size(0)
            total_samples += x.size(0)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss /= total_samples
        
        # 验证
        net.eval()
        valid_loss = 0
        total_samples = 0
        with torch.no_grad():
            for x, y in tqdm(valid_loader):
                x, y = x.to(device), y.to(device)
                preds = net(x)
                loss = criterion(preds, y)
                valid_loss += loss.item() * x.size(0)
                total_samples += x.size(0)
        valid_loss /= total_samples

        # 监测训练效果
        print("Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}".format(epoch+1, train_loss, valid_loss))
        
        # 记录训练效果
        train_loss_lst.append(train_loss)
        valid_loss_lst.append(valid_loss)
        
        # 更新本地模型
        if valid_loss < best_valid_loss * 0.999:
            best_valid_loss = valid_loss
            torch.save(net.state_dict(), model_path)
            print("Saved model with validation loss of {:.4f}".format(best_valid_loss)) 
            count = 0
        else:
            count += 1
            
        # 早停：若累计有patience次迭代，模型都没有进步，停止本轮训练
        if count >= patience:
            break
    
    
    # 记录当前训练轮次的指标变动，并更新本地储存结果
    results['round'].append(str(cnt))          
    results['train'].append(train_loss_lst)   
    results['valid'].append(valid_loss_lst)   
    with open(f'Models_v1/train_results_{cnt}.pickle', 'wb') as f:
        pickle.dump(results, f)
    
    # 下一轮
    cnt += 1

100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:11<00:00, 12.51it/s]


Epoch: 1, Training Loss: 0.0072, Validation Loss: 0.0075
Saved model with validation loss of 0.0075


100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:11<00:00, 12.66it/s]


Epoch: 2, Training Loss: 0.0017, Validation Loss: 0.0083


100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:11<00:00, 12.62it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 152/152 [00:12<00:00, 12.54it/s]


Epoch: 14, Training Loss: 0.0007, Validation Loss: 0.0260


100%|████████████████████████████████████████████████████████████████████████████████| 152/152 [00:12<00:00, 12.26it/s]


Epoch: 15, Training Loss: 0.0007, Validation Loss: 0.0076


 62%|██████████████████████████████████████████████████                               | 94/152 [00:07<00:04, 11.87it/s]

# 查看轮次对应的时间区间

In [ ]:
def get_time_range(round_idx):
    # 获取指定轮次的时间区间
    train_start_date = unique_dates[starts[round_idx]]
    valid_start_date = unique_dates[valid_starts[round_idx]]
    test_start_date = unique_dates[test_starts[round_idx]]
    test_end_date = unique_dates[ends[round_idx]]
    
    return {
        "train": (train_start_date, valid_start_date - timedelta(days=1)),
        "valid": (valid_start_date, test_start_date - timedelta(days=1)),
        "test": (test_start_date, test_end_date)
    }

In [ ]:
time_range = get_time_range(2)
print(f"""
训练集: {time_range['train'][0]} 至 {time_range['train'][1]}
验证集: {time_range['valid'][0]} 至 {time_range['valid'][1]}
测试集: {time_range['test'][0]} 至 {time_range['test'][1]}
""")